## EM Algorithm

In [107]:
import numpy as np
import math

In [108]:
new_data = np.loadtxt('em_data.txt')
print(new_data.shape)

(1000,)


In [109]:
#Initialize params

n = new_data.shape[0]
p_fp = 0.5
lambda1 = np.mean(new_data) + 10
lambda2 = np.mean(new_data) - 10

print(lambda1)
print(lambda2)

max_it = 1000
threshold = 1.0e-6
log_likelihood = None

13.797
-6.202999999999999


In [110]:
def poisson_dist(x, lambda_i):
    pow = lambda_i ** x
    expo = np.exp(-lambda_i)
    fact = math.factorial(int(x))
    return pow * expo / fact

def calc_log_likelihood(data, lambda1, lambda2, p_fp):   
    log_likelihood = 0
    for i in range(data.shape[0]):
        temp1 = p_fp * poisson_dist(data[i], lambda1)
        temp2 = (1 - p_fp) * poisson_dist(data[i], lambda2)

        log_likelihood += np.log(temp1 + temp2)
    return log_likelihood

In [111]:
def em_algo(data, lambda1, lambda2, p_fp, log_likelihood, max_it, threshold, epsilon):
    it_actual = 0
    for it in range(max_it):
        # print("Iteration: ", it)
        it_actual += 1
        #E-step
        p_lambda1_given_xi = np.zeros(n)
        # print(p_lambda1_given_xi.shape)
        # print(p_lambda1_given_xi[0])
        for i in range(n):
            p_xi_given_lambda1 = poisson_dist(data[i], lambda1) # P(Xi|lambda1)
            p_xi_given_lambda2 = poisson_dist(data[i], lambda2)

            # print("xi|lambdas:")
            # print(p_xi_given_lambda1)
            # print(p_xi_given_lambda2)

            temp1 = p_fp * p_xi_given_lambda1
            temp2 = (1 - p_fp) * p_xi_given_lambda2

            p_lambda1_given_xi[i] = temp1 / (temp1 + temp2) # P(lambda1|Xi)
            p_lambda1_given_xi = np.clip(p_lambda1_given_xi, epsilon, 1 - epsilon)
            # print("p_lambda1_given_xi:")
            # print(p_lambda1_given_xi)

        #M-step: update and optimize
        fp_new = np.sum(p_lambda1_given_xi) / n

        lambda1_new = np.sum(p_lambda1_given_xi * data) / np.sum(p_lambda1_given_xi + epsilon)
        lambda2_new = np.sum((1 - p_lambda1_given_xi) * data) / np.sum((1 - p_lambda1_given_xi) + epsilon)

        # abs1 = np.abs(lambda1_new - lambda1)
        # abs2 = np.abs(lambda2_new - lambda2)
        # abs3 = np.abs(fp_new - p_fp)

        # if abs1 < threshold and abs2 < threshold and abs3 < threshold:
        #     break

        log_likelihood_new = calc_log_likelihood(data, lambda1_new, lambda2_new, fp_new)
        # abs1 = np.abs(log_likelihood_new - log_likelihood)
        if log_likelihood is not None and (np.abs(log_likelihood_new - log_likelihood) < threshold):
            break

        log_likelihood = log_likelihood_new

        lambda1 = lambda1_new
        lambda2 = lambda2_new
        p_fp = fp_new

    return lambda1, lambda2, p_fp, it_actual

In [112]:
lambda1, lambda2, p_fp, it_actual = em_algo(new_data, lambda1, lambda2, p_fp, log_likelihood, max_it, threshold, 1.0e-6)

In [113]:
print("Iterations needed: ", it_actual)
print(" ")
print(f"Mean number of children in families with family planning: {lambda1:.2f}")
print(f"Mean number of children in families without family planning: {lambda2:.2f}")
print(" ")
print(f"Proportion of families with family planning: {p_fp:.2f}")
print(f"Proportion of families without family planning: {(1 - p_fp):.2f}")

Iterations needed:  155
 
Mean number of children in families with family planning: 1.78
Mean number of children in families without family planning: 4.91
 
Proportion of families with family planning: 0.36
Proportion of families without family planning: 0.64
